### Rooki to access WPS with climate data operations

In [ ]:
from rooki import rooki

In [ ]:
# The same as using birdy
#from birdy import WPSClient
#url = 'https://bovec.dkrz.de/ows/proxy/rook'
#url = 'http://localhost:5000/wps'
#rooki = WPSClient(url, verify=False)

### Available processes

In [ ]:
rooki?

### Run subset

In [ ]:
rooki.subset?

In [ ]:
resp_subset = rooki.subset(
    data_ref='cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas',
    time='2085-01-01/2120-12-30',
)

In [ ]:
resp_subset.get()

### Average process

In [ ]:
rooki.average?

### Run orchestrate workflow

In [ ]:
rooki.orchestrate?

**Workflow with subset and average**

In [ ]:
wf = {
    'doc': "subset workflow",
    'inputs': {
        'tas': ['cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas'],
    },
    "outputs": {
          "output": "average_tas/output"
      },
    'steps': {
        "subset_tas_1": {
              "run": "subset",
              "in": {
                  "data_ref": "inputs/tas",
                  "time": "2085-01-01/2120-12-30"
              }
          },
          "subset_tas_2": {
              "run": "subset",
              "in": {
                  "data_ref": "subset_tas_1/output",
                  "time": "2090-01-01/2100-12-30"
              }
          },
          "average_tas": {
              "run": "average",
              "in": {
                  "data_ref": "subset_tas_2/output",
                  "axes": "time"
              }
          }
    }}

In [ ]:
import json
result = rooki.orchestrate(workflow=json.dumps(wf))
result.get()

**Using operators for the same workflow**

In [ ]:
from rooki import operators as ops

wf = ops.Average(
    ops.Subset(
        ops.Subset(
            ops.Input(
                'tas', ['cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas'],
            ),
            time="2085-01-01/2120-12-30",
        ),
        time="2090-01-01/2100-12-30",
    ),
    axes='time',
)

In [ ]:
wf._serialise()

In [ ]:
out = wf.orchestrate()
out.get()

**Workflow with diff operator (multiple inputs)**

In [ ]:
wf = {
    'doc': "tree workflow with diff operator",
    'inputs': {
        'inm': ['cmip5.output1.INM.inmcm4.rcp45.mon.ocean.Omon.r1i1p1.latest.zostoga'],
        'mpi': ['cmip5.output1.MPI-M.MPI-ESM-LR.rcp45.mon.ocean.Omon.r1i1p1.latest.zostoga']
    },
    "outputs": {
          "output": "diff/output"
      },
    'steps': {
        "subsetA": {
              "run": "subset",
              "in": {
                  "data_ref": "inputs/inm",
                  "time": "2085-01-01/2120-12-30"
              }
          },
          "subsetB": {
              "run": "subset",
              "in": {
                  "data_ref": "inputs/mpi",
                  "time": "2085-01-01/2120-12-30"
              }
          },
          "diff": {
              "run": "diff",
              "in": {
                  "data_ref_a": "subsetA/output",
                  "data_ref_b": "subsetB/output",
              }
          }
    }}

In [ ]:
result = rooki.orchestrate(workflow=json.dumps(wf))
result.get()

**Same "diff" workflow with operators**

In [ ]:
wf = ops.Diff(
    ops.Subset(
        ops.Input(
            'inm', ['cmip5.output1.INM.inmcm4.rcp45.mon.ocean.Omon.r1i1p1.latest.zostoga']
        ),
        time="2085-01-01/2120-12-30",
    ),
    ops.Subset(
        ops.Input(
            'mpi', ['cmip5.output1.MPI-M.MPI-ESM-LR.rcp45.mon.ocean.Omon.r1i1p1.latest.zostoga']
        ),
        time="2085-01-01/2120-12-30",
    ),
)

In [ ]:
out = wf.orchestrate()
out.get()